In [1]:
import tensorflow as tf
import os
import fm
from sklearn.model_selection import train_test_split

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
tf.__version__

'2.1.0'

In [3]:
def preprocess(sent):
    sent = '<start> ' + sent + ' <end>'
    return sent
def create_dataset(path,num_examples):
    lines = fm.load_file(path)
    lines = [preprocess(s) for s in lines[:num_examples]]
    return lines
def max_length(tensor):
    return max(len(t) for t in tensor)
"""字典编码，返回一个张量和一个字典""" 
"""查询:tokenizer.index_word[index]""" 
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post') #在后方加padding
    return tensor, lang_tokenizer
def load_dataset(tgt_path,src_path,num_examples=None):
    tgt = list(create_dataset(tgt_path,num_examples))
    src = list(create_dataset(src_path,num_examples))
    tgt_tensor,tgt_tokenizer = tokenize(tgt)
    src_tensor,src_tokenizer = tokenize(src)
    return tgt_tensor,tgt_tokenizer,src_tensor,src_tokenizer

In [4]:
# 限制数据集大小
num_example = 100000
tgt_tensor,tgt_tokenizer,src_tensor,src_tokenizer = load_dataset('../../data/FBIS/fbis.zh','../../data/FBIS/fbis.en',num_example)

In [5]:
print(len(tgt_tensor))
max_len_tgt,max_len_src = max_length(tgt_tensor),max_length(src_tensor)
print("max length: target -",max_len_tgt,"source - ",max_len_src)

100000
max length: target - 4217 source -  483


In [6]:
# 划分训练集
src_train,src_val,tgt_train,tgt_val = train_test_split(src_tensor,tgt_tensor,test_size=0.2)
print(len(src_train),len(src_val),len(tgt_train),len(tgt_val))

80000 20000 80000 20000


In [7]:
def covert(lan,tensor):
    for t in tensor:
        if t!=0:
            print("%d ----> %s" % (t,lan.index_word[t]))

In [21]:
"""创建tf.data数据集"""
BUFFER_SIZE = len(src_train)
BATCH_SIZE = 64
steps_per_epoch = len(src_train)#BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(src_tokenizer.word_index)+1
vocab_tar_size = len(tgt_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((src_train, tgt_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)